## 3.6　手書き文字認識

In [1]:
import sys
import pickle
import numpy as np

In [5]:
sys.path.append('/Users/yuta.shimizu/Downloads/ML/deep-learning-from-scratch-master')

In [6]:
from dataset.mnist import load_mnist

In [32]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

### 3.6.1　MNIST データセット

In [23]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=False, flatten=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 1, 28, 28)
(60000,)
(10000, 1, 28, 28)
(10000,)


In [24]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=False, flatten=True)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


### 3.6.2　ニューラルネットワークの推論処理

In [25]:
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test
    
def init_network():
    with open('/Users/yuta.shimizu/Downloads/ML/deep-learning-from-scratch-master/ch03/sample_weight.pkl', 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [27]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1
        
print('Accuracy:' + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


### 3.6.3　バッチ処理

P.58 と P.78 を照らし合わせると、層の数とバッチサイズの区別が分かりやすい。<br>
784 個の層が入力され、10 個の層（0〜9 の各確率）が出力される。これが画像データ 1 枚に対応している。

In [28]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print(x.shape)
print(x[0].shape)
print(W1.shape)
print(W2.shape)
print(W3.shape)

(10000, 784)
(784,)
(784, 50)
(50, 100)
(100, 10)


`x[0]` はバッチサイズが 1 なので、1 個のニューラルネットワーク（784 層）を計算する。<br>
`x` はバッチサイズが 10000 なので、10000 個のニューラルネットワーク（784 層）を計算する。

バッチサイズ 100 でニューラルネットワークを実装する。

In [30]:
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p==t[i:i+batch_size])
    
print('Accuracy:' + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


In [33]:
x = np.array([[0.1, 0.8, 0.1], [0.3, 0.1, 0.6], [0.2, 0.5, 0.3], [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=1)
print(y)

[1 2 1 0]


In [34]:
t = np.array([1, 2, 0, 0])
print(t)

[1 2 0 0]


In [37]:
print(y==t)
print(np.sum(y==t))

[ True  True False  True]
3
